In [24]:
#Text to numbers
def text2int (textnum, numwords={}):
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]

        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

        scales = ["hundred", "thousand", "million", "billion", "trillion"]

        #numwords["and"] = (1, 0)
        for idx, word in enumerate(units):  numwords[word] = (1, idx)
        for idx, word in enumerate(tens):       numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ""
    onnumber = False
    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if word not in numwords:
                if onnumber:
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
            else:
                scale, increment = numwords[word]

                current = current * scale + increment
                if scale > 100:
                    result += current
                    current = 0
                onnumber = True

    if onnumber:
        curstring += repr(result + current)
    return curstring


#Testing
#test_str = "define Variable var1 and 'var2'. Add variables var1 from var2, Subtract variables var2 from var1, create a for loop and multiply var1 five times"
#text2int(test_str)

In [118]:
#Tokenize

import re

def tokenize_words(sentence):
    tokenize_statement=[]
    raw_string= "([ !@#$%^&*\(\)\-=+\[\]\{\};:'\"|\\<,.>/?])"
    tokenize_statement = re.split(r""+raw_string, sentence)
    return(tokenize_statement)

def tokenize_words_final(sentence):
    raw_input0 = text2int(sentence)
    raw_input1 = tokenize_words(raw_input0)
    #print(raw_input)

    raw_input2 = []
    for word in raw_input1:
        if word != ' ' and word != '':
            raw_input2.append(word)
    #print(raw_input2)
    return (raw_input2)

#cust_input = "define Variable var1 and 'var_2'. Add variables var1 from var2, Subtract variables var2 from var1, create a for loop and multiply var1 fifteen times"
#cust_input = "set text = \"hi\""
#cust_input = "print \"please select value\" + i"
#cust_input = input("Enter your value: ") 
#print(tokenize_words_final(cust_input))

In [119]:
#Read Files

raw_data=[]
with open("NLP_Data/Input_file.txt", "r") as input_file:
    for line in input_file:
        raw_data.append(line.strip())
#print (data)
data=[]
for each_line in raw_data:
    
    data.extend(tokenize_words_final(each_line))
    data.append("\n")

#print (data)

In [120]:
#Custom Tagging

import shelve
dest_file="NLP_Data/Shelf//" + "Dict_Lists" + ".shlf"
shelf = shelve.open(dest_file)

raw_datatypes = shelf["raw_datatypes"]
raw_operators = shelf["raw_operators"]
raw_commands = shelf["raw_commands"]
raw_loops = shelf["raw_loops"]
raw_loop_types = shelf["raw_loop_types"]

raw_equal = shelf["raw_equal"]
raw_scope = shelf["raw_scope"]

raw_others = shelf["raw_others"]
custom_stoppers = shelf["custom_stoppers"]
new_custom_class = shelf["custom_class"]
custom_puntuations = shelf["custom_puntuations"]

shelf.close()
def custom_tagging(data):
    tagged_string=""
    tag=""
    
    iter_data = iter(data)
    for each_word in iter_data:
        modified_each_word = each_word.lower()
        if modified_each_word in raw_datatypes:
            tag="D_TYPE"
        elif modified_each_word in raw_operators:
            tag="OPER"
        elif modified_each_word in raw_commands:
            tag="CMD"
        elif modified_each_word in raw_loops:
            tag="LOOP"      
        elif modified_each_word in raw_loop_types:
            tag="LP_TYPE"

        elif modified_each_word in raw_equal:
            tag="EQUAL"
        elif modified_each_word in raw_scope:
            tag="SCOPE"
        elif modified_each_word in new_custom_class:
            tag="CUST_CLASS"

        elif modified_each_word.isnumeric():
            tag="AMNT"
        elif modified_each_word == "to":
            tag="TO"
        elif modified_each_word == "and":
            tag="CC"
        elif modified_each_word in raw_others:
            tag="OTHER"    


        elif modified_each_word =="\n":
            tag=""
        elif modified_each_word == "\"" or each_word == "'":
            tag= "\"\"\""  

            #This is for tagging all the indide of " and ' as values
            flag=False
            curr_word = each_word
            while flag != True:
                #print (each_word+"\t"+tag)
                tagged_string += each_word+"\t"+tag+"\n"
                #tagged_string.append([each_word,tag])
                each_word = next(iter_data, None)
                tag = "VALUE"

                if each_word==curr_word:
                    tag="\"\"\""  
                    flag=True
            #END 

        elif modified_each_word == "." :
            tag= "."
        elif modified_each_word == "," :
            tag= ","
        else:
            tag="VAR"   

            
        if modified_each_word == "\n":
            #print()
            tagged_string += "\n"
            #tagged_string.append("\n")
        else:
            #print (each_word+"\t"+tag)
            tagged_string += each_word+"\t"+tag+"\n"
            #tagged_string.append([each_word,tag])
            
    return (tagged_string)

print(custom_tagging(data))

define	CMD
global	SCOPE
variable	D_TYPE

create	CMD
global	SCOPE
variable	D_TYPE

define	CMD
variable	D_TYPE

create	CMD
variable	D_TYPE

set	CMD
variable	D_TYPE
equal	EQUAL
to	TO
10	AMNT

set	CMD
variable	D_TYPE
i	VAR
=	EQUAL
10	AMNT

define	CMD
variable	D_TYPE
i	VAR

create	CMD
variable	D_TYPE
i	VAR

set	CMD
variable	D_TYPE
equal	EQUAL
I	VAR
to	TO
0	AMNT

set	CMD
variable	D_TYPE
i	VAR
=	EQUAL
0	AMNT

define	CMD
variable	D_TYPE
j	VAR

create	CMD
variable	D_TYPE
j	VAR

set	CMD
variable	D_TYPE
equal	EQUAL
j	VAR
to	TO
0	AMNT

set	CMD
variable	D_TYPE
j	VAR
=	EQUAL
0	AMNT

set	CMD
text	VAR
=	EQUAL
"	"""
hi	VALUE
"	"""

define	CMD
variable	D_TYPE
text1	VAR

define	CMD
variable	D_TYPE
text1	VAR
=	EQUAL
"	"""
please	VALUE
enter	VALUE
value	VALUE
"	"""

set	CMD
variable	D_TYPE
text1	VAR
=	EQUAL
"	"""
please	VALUE
enter	VALUE
value	VALUE
"	"""

set	CMD
text1	VAR
=	EQUAL
"	"""
please	VALUE
enter	VALUE
value	VALUE
"	"""

print	CMD
text1	VAR

print	CMD
"	"""
please	VALUE
select	VALUE
value	VALUE
"

In [121]:
#Write Files
filename = "NLP_Data/Output_file.txt"

with open(filename, "w") as output_file:
    store_data = custom_tagging(data)
    for line in store_data:
        output_file.write(line)
'''
#Write in CSV
import csv
with open(filename, "w") as output_file:
    writer =  csv.writer(output_file)
    store_data = custom_tagging(data)
    for line in store_data:
        writer.writerow(line)
'''
print(end="")

In [91]:
#Detect Variable (Incomplete)
def variable_detect(word):
    tokenize_statement=[]
    raw_string= "^[A-Za-z]+_*[0-9]+$"
    tokenize_statement = re.split(r""+raw_string, word)
    if word == tokenize_statement:
        print (tokenize_statement)
    else:
        print (tokenize_statement)
    #return(tokenize_statement)
for word in tokenize_words_final(cust_input):
    variable_detect(word)

['define']
['Variable']
['', '']
['and']
["'"]
['', '']
["'"]
['.']
['Add']
['variables']
['', '']
['from']
['', '']
[',']
['Subtract']
['variables']
['', '']
['from']
['', '']
[',']
['create']
['a']
['for']
['loop']
['and']
['multiply']
['', '']
['15']
['times']
